In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.decomposition import PCA  #次元削減用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR
import time
import datetime
import os #ディレクトリ作成用
import xgboost as xgb
import sys


from sklearn.preprocessing import StandardScaler#モデルの評価用に標準化する関数
import scipy.stats#モデルの評価用に標準化する関数

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

In [3]:
#モデルスコアデータの読み込み
score_df = pd.read_csv('ex_score.csv')
score_df=score_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
display(score_df)

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,256020.0,314900.0,234.0,3149.0,81.302001,7.43093,234.0
1,1.0,4.0,52.0,256020.0,314900.0,234.0,3149.0,81.302001,7.43093,234.0
2,1.0,4.0,53.0,256020.0,314900.0,234.0,3149.0,81.302001,7.43093,234.0
3,1.0,4.0,54.0,256020.0,314900.0,234.0,3149.0,81.302001,7.43093,234.0
4,1.0,4.0,55.0,256020.0,314900.0,234.0,3149.0,81.302001,7.43093,234.0
...,...,...,...,...,...,...,...,...,...,...
57910,119.0,8.0,145.0,0.0,26400.0,1.0,264.0,0.000000,0.00000,0.0
57911,119.0,8.0,146.0,0.0,9700.0,1.0,97.0,0.000000,0.00000,0.0
57912,119.0,8.0,147.0,0.0,9700.0,1.0,97.0,0.000000,0.00000,0.0
57913,119.0,8.0,148.0,0.0,9700.0,1.0,97.0,0.000000,0.00000,0.0


#### モデル性能評価用のデータフレーム作成(result_comごとの閾値を持つ)


In [7]:
"""
#学習データの読み込み
result_df = pd.read_csv('../csv/train_asiya.csv')
result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除

df=result_df
#学習、テストデータ切り分け
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
display(result_com_df)
"""

'\n#学習データの読み込み\nresult_df = pd.read_csv(\'../csv/train_asiya.csv\')\nresult_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除\n\ndf=result_df\n#学習、テストデータ切り分け\nnum_data = len(df) \nnum_valid = int(num_data/6.0)\nvalid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに\ntrain_df = df.loc[0:(num_data-num_valid-1), :]\ntrain_money=pd.Series(train_df[\'money\'])\nvalid_money=pd.Series(valid_df[\'money\'])\n#出現数の分布\nresult_com_s=valid_df[\'result_com\'].value_counts()\nresult_com_s=result_com_s.sort_index()\ngain_mean=valid_df.groupby(\'result_com\')[\'money\'].mean()\ngain_mean=gain_mean.sort_index()\n\ngain_median=valid_df.groupby(\'result_com\')[\'money\'].median()\ngain_median=gain_median.sort_index()\nresult_com_df=pd.DataFrame({\'result_com\':result_com_s.index,\n                            \'result_com_num\':result_com_s.values,\n                            \'result_com_per\':result_com_s.values/sum(result_com_s.values)*100,\n                            \'gai

### モデル性能評価機能

In [8]:
#基準をクリアしたモデルを格納するdf
#depths_arr=[5,6,8]
depths_arr=[4,5,6,7,8]
result_com_arr=np.arange(1, 29)
#th_arr=[]#未定探索終わり次第なところはある。

#基本敵に今回からは出現数とかはresult_comのdfから作成しないでスコアのdfからしゅとくする。

#model_para_df=pd.DataFrame(columns=['target_com','depth','target_per',threshold,'総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
model_para_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for target_com in result_com_arr:
    target_com_df=score_df[score_df['target_com']==target_com]
    num_com= target_com_df['出現数'].values[0]  #出現数
    #閾値の作成===============================================================
    #gain_th=110#利益率の閾値(メインモデル)
    gain_th=115#利益率の閾値(メインモデル)
    #result_s=result_com_df[result_com_df['result_com']==target_com]
    buy_accuracy_th=5#result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値(いったん決めで5%とする)
    num_tp_th=num_com*0.15#あたった回数の閾値(出現回数の15%が的中)
    
    diff_gain_th=105#利益率の閾値(前後モデル)
    diff_buy_accuracy_th=5#result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値(いったん決めで5%とする)
    diff_num_tp_th=num_com*0.15#あたった回数の閾値(出現回数の15%が的中)
    
    #==========================================================
    for target_depth in depths_arr:
        target_com_depth_df=target_com_df[target_com_df['depth']==target_depth]
        
        len_df=len(target_com_depth_df)
        df=target_com_depth_df.copy()
        df['number_i']=np.arange(0,len_df,1)
        #for th in th_arr:
        for _, row in df.iterrows():
            #if ((row['number_i']==0) or (row['number_i']==1) or (row['number_i']==len_df-2) or (row['number_i']==len_df-1)):
            if ((row['number_i']==0) or (row['number_i']==len_df-1)):#前後モデルが存在しないのは評価に含めない（）
                pass
            else:
                #前後のモデルのスコア(今回から±１にした)===================================
                #diff_m2_row=df[df['number_i']==(row['number_i']-2)]
                diff_m1_row=df[df['number_i']==(row['number_i']-1)]
                diff_p1_row=df[df['number_i']==(row['number_i']+1)]
                #diff_p2_row=df[df['number_i']==(row['number_i']+2)]
                #diff_models=[diff_m2_row,diff_m1_row,diff_p1_row,diff_p2_row]
                diff_models=[diff_m1_row,diff_p1_row]
                #===================================================
                if ((row['利益率']>gain_th) and (row['購買的中率']>buy_accuracy_th) and (row['的中数']>=num_tp_th)):#初めに真ん中のモデルの性能評価
                    #前後モデルの評価
                    flag_arr=[0,0]
                    for index in range(len(flag_arr)):

                        diff_model=diff_models[index].iloc[0,:]
                        if ((diff_model['利益率']>diff_gain_th) and (diff_model['購買的中率']>diff_buy_accuracy_th) and (diff_model['的中数']>=diff_num_tp_th)):#前後モデルの性能評価
                            flag_arr[index]=1
                        else:
                            pass
                    if sum(flag_arr)==2:
                        model_para_df=model_para_df.append(row)
                        
                else:
                    pass
model_para_df.to_csv('good_score_model.csv', encoding='utf_8_sig')

### モデル性能の絶対正答数とgainを標準化したものをスコアとして　
### 一つのcomで一つのモデルを厳選する

In [12]:
good_model_df=pd.read_csv('good_score_model.csv')
good_model_df=good_model_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
good_model_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
0,3.0,4.0,116.0,21240.0,17100.0,145.0,171.0,124.210526,16.374269,28.0,65.0
1,3.0,4.0,117.0,21700.0,14700.0,145.0,147.0,147.619048,18.367347,27.0,66.0
2,3.0,4.0,118.0,15380.0,13000.0,145.0,130.0,118.307692,16.923077,22.0,67.0
3,3.0,5.0,118.0,23950.0,20600.0,145.0,206.0,116.262136,14.563107,30.0,67.0
4,3.0,5.0,119.0,21790.0,18800.0,145.0,188.0,115.904255,14.893617,28.0,68.0
5,3.0,5.0,120.0,20650.0,16300.0,145.0,163.0,126.687117,16.564417,27.0,69.0
6,3.0,5.0,121.0,17180.0,13900.0,145.0,139.0,123.597122,16.546763,23.0,70.0
7,3.0,6.0,121.0,24350.0,20300.0,145.0,203.0,119.950739,15.763547,32.0,70.0
8,3.0,6.0,125.0,20610.0,15100.0,145.0,151.0,136.490066,17.880795,27.0,74.0
9,3.0,6.0,126.0,17910.0,14100.0,145.0,141.0,127.021277,17.730496,25.0,75.0


In [13]:
good_model_df[good_model_df['target_com']==7]

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
24,7.0,5.0,137.0,16480.0,13600.0,139.0,136.0,121.176471,20.588235,28.0,86.0
25,7.0,7.0,146.0,20940.0,18200.0,139.0,182.0,115.054945,20.329670,37.0,95.0


In [18]:
for com in sorted(good_model_df['target_com'].value_counts().index):
    target_com_df=good_model_df[good_model_df['target_com']==com]
    #sc = StandardScaler()
    #sc =sc.fit(target_com_df['的中数'].values)
    target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
    #target_com_df['的中数_std']=sc.transform(target_com_df['的中数'].values)
    target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
    #target_com_df['利益率_std']=sc.fit_transform(target_com_df['利益率'].values)
    target_com_df['score']=target_com_df['的中数_std']+target_com_df['利益率_std']
    if len(target_com_df)>=3:
        use_model_df=target_com_df[target_com_df['score']==target_com_df['score'].max()]
    elif len(target_com_df)==2:#モデルが二つしかなかった時、最良のモデルが二つできてまう。
        use_model_df=target_com_df[target_com_df['利益率']==target_com_df['利益率'].max()]#そんなときは利益率で選ぶ
    elif len(target_com_df)==1:#モデルが1つしかなかった時、そのまま代入
        use_model_df=target_com_df
    
    display(use_model_df)
    #display(target_com_df)

<ipython-input-18-068089e98c7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-18-068089e98c7f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-18-068089e98c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i,的中数_std,利益率_std,score
1,3.0,4.0,117.0,21700.0,14700.0,145.0,147.0,147.619048,18.367347,27.0,66.0,0.235702,1.907789,2.143491


<ipython-input-18-068089e98c7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-18-068089e98c7f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-18-068089e98c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i,的中数_std,利益率_std,score
20,4.0,7.0,130.0,34030.0,27700.0,89.0,277.0,122.851986,10.830325,30.0,79.0,0.841191,1.358648,2.19984


<ipython-input-18-068089e98c7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-18-068089e98c7f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-18-068089e98c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

...

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i,的中数_std,利益率_std,score
24,7.0,5.0,137.0,16480.0,13600.0,139.0,136.0,121.176471,20.588235,28.0,86.0,-1.0,1.0,2.220446e-15


<ipython-input-18-068089e98c7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-18-068089e98c7f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-18-068089e98c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i,的中数_std,利益率_std,score
27,13.0,7.0,124.0,44140.0,30800.0,84.0,308.0,143.311688,6.493506,20.0,73.0,-1.0,1.0,-2.220446e-15


<ipython-input-18-068089e98c7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['的中数_std']=scipy.stats.zscore(target_com_df['的中数'].values)
<ipython-input-18-068089e98c7f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_com_df['利益率_std']=scipy.stats.zscore(target_com_df['利益率'].values)
<ipython-input-18-068089e98c7f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i,的中数_std,利益率_std,score
36,14.0,7.0,137.0,26050.0,19600.0,79.0,196.0,132.908163,9.183673,18.0,86.0,1.126357,0.774619,1.900976


In [17]:
target_com_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
0,3.0,4.0,116.0,21240.0,17100.0,145.0,171.0,124.210526,16.374269,28.0,65.0
1,3.0,4.0,117.0,21700.0,14700.0,145.0,147.0,147.619048,18.367347,27.0,66.0
2,3.0,4.0,118.0,15380.0,13000.0,145.0,130.0,118.307692,16.923077,22.0,67.0
3,3.0,5.0,118.0,23950.0,20600.0,145.0,206.0,116.262136,14.563107,30.0,67.0
4,3.0,5.0,119.0,21790.0,18800.0,145.0,188.0,115.904255,14.893617,28.0,68.0
5,3.0,5.0,120.0,20650.0,16300.0,145.0,163.0,126.687117,16.564417,27.0,69.0
6,3.0,5.0,121.0,17180.0,13900.0,145.0,139.0,123.597122,16.546763,23.0,70.0
7,3.0,6.0,121.0,24350.0,20300.0,145.0,203.0,119.950739,15.763547,32.0,70.0
8,3.0,6.0,125.0,20610.0,15100.0,145.0,151.0,136.490066,17.880795,27.0,74.0
9,3.0,6.0,126.0,17910.0,14100.0,145.0,141.0,127.021277,17.730496,25.0,75.0


In [6]:
model_para_df

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数,number_i
1258,3.0,6.0,121.0,24350.0,20300.0,145.0,203.0,119.950739,15.763547,32.0,70.0
1862,4.0,7.0,131.0,31920.0,28300.0,89.0,283.0,112.791519,10.247350,29.0,80.0
5999,13.0,4.0,110.0,76970.0,59000.0,84.0,590.0,130.457627,5.084746,30.0,59.0
6000,13.0,4.0,111.0,61030.0,52800.0,84.0,528.0,115.587121,5.303030,28.0,60.0
6096,13.0,5.0,108.0,109840.0,89600.0,84.0,896.0,122.589286,4.687500,42.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...
56520,117.0,4.0,141.0,57400.0,38000.0,3.0,380.0,151.052632,0.263158,1.0,90.0
56616,117.0,5.0,138.0,57400.0,48400.0,3.0,484.0,118.595041,0.206612,1.0,87.0
56617,117.0,5.0,139.0,57400.0,48400.0,3.0,484.0,118.595041,0.206612,1.0,88.0
56618,117.0,5.0,140.0,57400.0,48700.0,3.0,487.0,117.864476,0.205339,1.0,89.0


In [7]:
model_para_df.to_csv('../csv/good_score_model.csv', encoding='utf_8_sig')

In [33]:
diff_model

(1047, target_com        3.000000
 depth             4.000000
 target_per      108.000000
 総収益           38850.000000
 投資金額          46200.000000
 出現数             145.000000
 購買予測数           462.000000
 利益率              84.090909
 購買的中率             9.523810
 的中数              44.000000
 number_i         57.000000
 Name: 1047, dtype: float64)

In [17]:
len_df

99

In [34]:
type(diff_model)

tuple